# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts : the style of creating inputs to pass into the models. 
   * Models : language models
   * Output parsers : It involves taking the output of these models and parsing it into a more structured forma

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [1]:
# Set up environment
# !pip install python-dotenv
# !pip install openai
# !pip install --upgrade langchain

     |████████████████████████████████| 73 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 359 kB 3.0 MB/s eta 0:00:01     |██████████                      | 112 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 124 kB 1.3 MB/s eta 0:00:01


In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]


In [3]:
customer_email = """
我打冰沙的時候，蓋子不小心飛出去 \
冰沙全部噴出來，濺到牆上 \
更慘的是\
機器保證書上並不包含清潔廚房\
老友 \
偶需要你幫忙
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

response = get_completion(prompt)
response

## Chat API : LangChain

Let's try how we can do the same using LangChain.

Why not just use f"string" but using "prompt_template" in LangChain?
##### As you build sophisticated applications,  prompts can be quite long and detailed. And so, "prompt templates" are a useful abstraction to help you reuse good prompts when you can. 


In [8]:
# Model

    # To control the randomness and creativity of the generated
    # text by an LLM, use temperature = 0.0
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0.0)


# Prompt template
template_string = prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)
customer_messages = prompt_template.format_messages(
                    style=style,
                    text=customer_email)

# Call the model
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)
print(customer_response.content)

# Prompt Template

In [4]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [5]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True)

In [6]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [9]:
# Using another example with LangChain

service_reply = """哈囉妳好 客戶兒, \
我們的保固不包含清理服務 \
這不是基本常識ㄇ？ \
是你自己亂用果汁機 還忘記蓋蓋子 \
請記取教訓 \
掰掰啦 \
再會
"""

service_style_pirate = """\
a polite and respecful tone \
that speaks in English \
"""

service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

service_response = chat(service_messages)
print(service_response.content)

Translate the text that is delimited by triple backticks into a style that is a polite and respecful tone that speaks in English . text: ```哈囉妳好 客戶兒, 我們的保固不包含清理服務 這不是基本常識ㄇ？ 是你自己亂用果汁機 還忘記蓋蓋子 請記取教訓 掰掰啦 再會
```

Hello, dear customer. Our warranty does not cover cleaning services. Isn't this common knowledge? It seems that you have used the juicer improperly by forgetting to put on the lid. Please take this as a lesson learned. Goodbye and take care. See you again.


In [10]:
type(service_response.content)

str

## Output Parsers

The output from LLM model is a string, and we want to change it to JSON type later on 

In [11]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [13]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [15]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)
messages = prompt_template.format_messages(text=customer_review)
print('message', messages)

# create Langchain Model
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]
message [HumanMessage(content="For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many

In [16]:
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [17]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [18]:
# Define 3 types of schemas and parse them into a list
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [19]:
# having the output parser tell you what instructions it wants you to send to the LLM.
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [20]:
# create 'format instructions' variable, it includes precise set of instructions for the LLM that will cause it to generate an output that the output parser can process.
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [21]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)
print(messages[0].content)

# call LLM model
response = chat(messages)
print(response.content)
print("type: ", type(response.content))

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [22]:
# Call output_parser to convert string into json
output_dict = output_parser.parse(response.content)
print(type(output_dict))
output_dict


<class 'dict'>


{'gift': False,
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [23]:
output_dict.get('delivery_days')

'2'